# Early testing of the JUST algorithms decscribed in Ghaderpour 2020
* 02/13/23
* 02/14/23


In [6]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np
import json


# local modules 
import sys
sys.path.insert(0, "../../JUST")
from JUSTjumps import *

sys.path.insert(0, "../scripts")
from helpers import *


## import time series 

In [7]:
df = pd.read_csv("../constants/td_ambient_102022.csv" )
df["DateTime"] = pd.to_datetime(df["DateTime"])
df.head()

,DateTime,Temp C,RH %,Room,Ambient Temp,Ambient RH,Window Open
0,2022-07-20 07:20:00,23.189,51.268,0,14.333333,30.555556,1
1,2022-07-20 07:20:30,23.189,51.362,0,14.333333,30.555556,1
2,2022-07-20 07:21:00,23.189,51.488,0,14.333333,30.555556,1
3,2022-07-20 07:21:30,23.189,51.457,0,14.333333,30.555556,1
4,2022-07-20 07:22:00,23.165,51.517,0,14.333333,30.555556,1


In [8]:
dftime = df.copy()

# transform datetime index to actual pandas datetime, and set as index of dataframe 
dftime["DateTime"] = pd.to_datetime(dftime["DateTime"])
dftime.set_index("DateTime", inplace=True)

In [9]:
dftime

,Temp C,RH %,Room,Ambient Temp,Ambient RH,Window Open
DateTime,,,,,,
2022-07-20 07:20:00,23.189,51.268,0,14.333333,30.555556,1
2022-07-20 07:20:30,23.189,51.362,0,14.333333,30.555556,1
2022-07-20 07:21:00,23.189,51.488,0,14.333333,30.555556,1
2022-07-20 07:21:30,23.189,51.457,0,14.333333,30.555556,1
2022-07-20 07:22:00,23.165,51.517,0,14.333333,30.555556,1
...,...,...,...,...,...,...
2022-09-11 07:58:00,29.196,50.861,1,19.611111,30.000000,0
2022-09-11 07:58:30,29.196,50.861,1,19.611111,30.000000,0
2022-09-11 07:59:00,29.196,50.796,1,19.611111,30.000000,0


In [10]:
# type(exp_b_end)

In [11]:
with open('../constants/window_treatment.json') as f:    
    window_treatment = json.load(f)

exp_a_end = str2dt(window_treatment["072522"]["cutoff_times"]["end"])
exp_b_end = str2dt(window_treatment["081622"]["cutoff_times"]["end"])

# only consider experiment a 
df_a = df.loc[df["DateTime"] < exp_a_end]
df_a.head()

,DateTime,Temp C,RH %,Room,Ambient Temp,Ambient RH,Window Open
0,2022-07-20 07:20:00,23.189,51.268,0,14.333333,30.555556,1
1,2022-07-20 07:20:30,23.189,51.362,0,14.333333,30.555556,1
2,2022-07-20 07:21:00,23.189,51.488,0,14.333333,30.555556,1
3,2022-07-20 07:21:30,23.189,51.457,0,14.333333,30.555556,1
4,2022-07-20 07:22:00,23.165,51.517,0,14.333333,30.555556,1


In [12]:
# type(pd.datetime(exp_a_end))

In [13]:
df['DateTime']

0        2022-07-20 07:20:00
1        2022-07-20 07:20:30
2        2022-07-20 07:21:00
3        2022-07-20 07:21:30
4        2022-07-20 07:22:00
                 ...        
123601   2022-09-11 07:58:00
123602   2022-09-11 07:58:30
123603   2022-09-11 07:59:00
123604   2022-09-11 07:59:30
123605   2022-09-11 08:00:00
Name: DateTime, Length: 123606, dtype: datetime64[ns]

In [14]:
mask = (df['DateTime'] > exp_a_end) & (df['DateTime'] > exp_b_end)

In [15]:
# split dataframe into two dataframes, one for each room 
df1, df0 = [x.reset_index(drop=True) for _, x in df_a.groupby(df_a['Room'] < 1)]

In [16]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df0["DateTime"],
    y=df0["Temp C"], 
    name="Room 0 Temp",
    mode='lines+markers',
))

fig.update_layout(xaxis_title='Dates',
                  yaxis_title='Temperature (ºC)',)

## trim and filter data 01

In [17]:
# only take the first day of data 
noon_start = str2dt('2022, 07, 21, 00, 00')
mask = df0['DateTime'] < noon_start
df01 = df0.loc[mask].reset_index(drop=True)


# filter data to hourly 
df02 = make_df_with_freq(df01, "60T")

# get the time indices 
ts = [ts[1] for ts in list(df02.index)]
ts = pd.to_datetime(ts).hour 

In [18]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=ts,
    y=df02["Temp C"], 
    name="Room 0 Temp",
    mode='lines+markers',
))

fig.update_layout(xaxis_title='Times',
                  yaxis_title='Temperature (ºC)',)

In [19]:
ix, mag, direction = JumpDetect(ts, df02["Temp C"])
df02.iloc[ix]

Temp C          24.897400
RH %            49.426483
Room             0.000000
Ambient Temp    20.500000
Ambient RH      18.888889
Window Open      1.000000
Name: (0, 2022-07-20 13:00:00), dtype: float64

## just filter, exp01

In [20]:
# filter data to hourly 
df03 = make_df_with_freq(df0, "60T")

# get the time indices 
ts3 = [ts3[1] for ts3 in list(df03.index)]
# ts03 = pd.to_datetime(ts3).day + pd.to_datetime(ts3).hour/100
# ts3

In [24]:
ts3

[Timestamp('2022-07-20 07:00:00'),
 Timestamp('2022-07-20 08:00:00'),
 Timestamp('2022-07-20 09:00:00'),
 Timestamp('2022-07-20 10:00:00'),
 Timestamp('2022-07-20 11:00:00'),
 Timestamp('2022-07-20 12:00:00'),
 Timestamp('2022-07-20 13:00:00'),
 Timestamp('2022-07-20 14:00:00'),
 Timestamp('2022-07-20 15:00:00'),
 Timestamp('2022-07-20 16:00:00'),
 Timestamp('2022-07-20 17:00:00'),
 Timestamp('2022-07-20 18:00:00'),
 Timestamp('2022-07-20 19:00:00'),
 Timestamp('2022-07-20 20:00:00'),
 Timestamp('2022-07-20 21:00:00'),
 Timestamp('2022-07-20 22:00:00'),
 Timestamp('2022-07-20 23:00:00'),
 Timestamp('2022-07-21 00:00:00'),
 Timestamp('2022-07-21 01:00:00'),
 Timestamp('2022-07-21 02:00:00'),
 Timestamp('2022-07-21 03:00:00'),
 Timestamp('2022-07-21 04:00:00'),
 Timestamp('2022-07-21 05:00:00'),
 Timestamp('2022-07-21 06:00:00'),
 Timestamp('2022-07-21 07:00:00'),
 Timestamp('2022-07-21 08:00:00'),
 Timestamp('2022-07-21 09:00:00'),
 Timestamp('2022-07-21 10:00:00'),
 Timestamp('2022-07-

In [21]:
def normalize_scale(arr, tmin, tmax):
    "normalize the values in an array to be between 0 and 1 based on the minimum and maximum values in the array"
    arr2 =  (arr - arr.min())/(arr.max() - arr.min())  * (tmax - tmin) + tmin
    return arr2

In [22]:
ts03 = pd.to_datetime(ts3).day  + normalize_scale(pd.to_datetime(ts3).hour, 0.0000001, 0.9999999)

In [23]:
ts03

Float64Index([ 20.30434786521739, 20.347826117391303,  20.39130436956522,
               20.43478262173913, 20.478260873913044, 20.521739126086956,
               20.56521737826087, 20.608695630434784, 20.652173882608697,
               20.69565213478261,  20.73913038695652, 20.782608639130434,
               20.82608689130435, 20.869565143478262, 20.913043395652174,
              20.956521647826087,         20.9999999,         21.0000001,
              21.043478352173913, 21.086956604347826, 21.130434856521738,
               21.17391310869565, 21.217391360869566,  21.26086961304348,
               21.30434786521739, 21.347826117391303,  21.39130436956522,
               21.43478262173913, 21.478260873913044, 21.521739126086956,
               21.56521737826087, 21.608695630434784, 21.652173882608697,
               21.69565213478261,  21.73913038695652, 21.782608639130434,
               21.82608689130435, 21.869565143478262, 21.913043395652174,
              21.956521647826087,     

In [88]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=ts03,
    y=df03["Temp C"], 
    name="Room 0 Temp",
    mode='lines+markers',
    text=ts3
))

fig.update_layout(xaxis_title='Normalized Dates and Times',
                  yaxis_title='Temperature (ºC)',)

In [76]:
# just find one jump 
ix, mag, direction = JumpDetect(ts03, df03["Temp C"])
df03.iloc[ix]

Temp C          24.939567
RH %            50.341883
Room             0.000000
Ambient Temp    22.333333
Ambient RH      16.111111
Window Open      1.000000
Name: (0, 2022-07-22 13:00:00), dtype: float64

In [81]:
# look at all jumps 
res = AllJumps(ts03, df03["Temp C"])
res

[[54, 0.5506474492739599, -0.09076444333432107, 18.5],
 [76, -0.6663533713252328, 0.42350910897018856, 16.5],
 [76, -0.711061727327202, 0.4799985916247403, 15.5]]

In [107]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=ts3,
    y=df03["Temp C"], 
    name="Room 0 Temp",
    mode='lines+markers',
    text=ts03
))


for i in res:
    fig.add_trace(go.Scatter(
    x=[df03.iloc[i[0]].name[1]],
    y=[df03.iloc[i[0]]["Temp C"]], 
    name=f"Jump Detect Mag: {np.round(i[1],2)}",
    mode='lines+markers',
    marker_color="red",
    text=f"{i}"
    ))


fig.update_layout(xaxis_title='Times',
                  yaxis_title='Temperature (ºC)',
                  title="Unmodified Data")